## Custom API Notebook

### Initialize the Node

In [1]:
import syft as sy
import pandas as pd
from typing import Any
from syft import SyftSuccess

node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_client.register(
    email="user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Staging Protocol Changes...
Starting test-domain-1 server on 0.0.0.0:19497

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=54ff726fa8ab40bbb6f4a74fd44852cc in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


INFO:     Started server process [348636]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:19497 (Press CTRL+C to quit)


INFO:     127.0.0.1:40842 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:40856 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:40856 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Waiting for server to start Done.
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:40856 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:40856 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:40872 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

INFO:     127.0.0.1:40856 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:40874 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:40874 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:40874 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:40874 - "GET /api/v2/api?verify_key=a23cabbbfad102773f54fba97cfcc97db0dfa0a6799c8abfd7e46f1ed40a697d&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:40888 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <user@openmined.org>
INFO:     127.0.0.1:40890 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:40892 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:55002 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:55002 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:55002 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:55002 - "GET /api/v2

### Create a public custom API Endpoint by using the decorator

This allows node admin to create a new public endpoint by using only the decorator.

In [2]:
@sy.api_endpoint(
    path="first.query",
    settings={"key": "value"},
)
def public_endpoint_method(
    context,
    query: str,
) -> Any:
    return context.settings['key'] == "value"


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=public_endpoint_method)
assert isinstance(response, SyftSuccess)

In [3]:
assert len(domain_client.api.services.api.api_endpoints()) == 1

In [4]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <user@openmined.org>


In [7]:
assert domain_client.api.services.first.query(query="SELECT *")
assert domain_guest.api.services.first.query(query="SELECT *")

### Create public/private Custom API Endpoint using TwinAPIEndpoint

This allows the admin to create a public/private endpoint interface where the users can iteract with.

In [5]:
@sy.public_api_endpoint(
    settings={"Hello": "Public"}
)
def public_function(
    context,
) -> str:
    return "Public Function Execution"

@sy.private_api_endpoint(
    settings={"Hello": "Private"}
)
def private_function(
    context,
) -> str:
    return "Private Function Execution"

new_endpoint = sy.TwinAPIEndpoint(
    path="third.query",
    private=private_function,
    public=public_function,
    description="Lore ipsulum ...",
)

# # Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_endpoint)

SyftSuccess: Endpoint successfully created.

In [ ]:
assert isinstance(response, SyftSuccess)
assert len(domain_client.api.services.api.api_endpoints()) == 2

In [8]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <user@openmined.org>


In [9]:
assert domain_client.api.services.third.query() == "Private Function Execution"

In [10]:
assert domain_guest.api.services.third.query() == "Public Function Execution"

In [11]:
@sy.syft_function_single_use(
    endpoint=domain_client.api.services.third.query,
)
def job_function(endpoint):
    return endpoint()

# Create a new project
new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[domain_client],
)

result = new_project.create_code_request(job_function, domain_client)
assert isinstance(result, SyftSuccess)

SyftSuccess: Syft function 'job_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [13]:
domain_client.requests[-1].approve()

Approving request for domain test-domain-1


SyftSuccess: Request d5efce93e1b64cf38247bd5beb583c9a changes applied

In [15]:
domain_client.code.job_function(endpoint=domain_client.api.services.third.query)

```python
Pointer
```
'Private Function Execution'